In [1]:
from eda import *

In [2]:
downloads = downloads.cache()

In [3]:
ibis.options.interactive = True

In [4]:
downloads.filter(ibis._.timestamp > datetime.now() - timedelta(days=30)).group_by(
    ibis._.system
).agg(ibis._.downloads.sum().name("downloads")).order_by(ibis._.downloads.desc())

┏━━━━━━━━━┳━━━━━━━━━━━┓
┃ system  ┃ downloads ┃
┡━━━━━━━━━╇━━━━━━━━━━━┩
│ string  │ int64     │
├─────────┼───────────┤
│ Linux   │     62385 │
│ NULL    │     17058 │
│ Windows │       461 │
│ Darwin  │       342 │
└─────────┴───────────┘

In [5]:
t = (
    downloads.group_by(
        [ibis._.timestamp.truncate("M").name("timestamp"), ibis._.version]
    )
    .agg(ibis._.downloads.sum().name("downloads"))
    .order_by(ibis._.timestamp.desc())
    .mutate(
        ibis._.downloads.sum()
        .over(rows=(0, None), group_by="version", order_by=ibis._.timestamp.desc())
        .name("total_downloads")
    )
    .order_by(ibis._.timestamp.desc())
)

In [6]:
t

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ timestamp           ┃ version      ┃ downloads ┃ total_downloads ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ timestamp           │ string       │ int64     │ int64           │
├─────────────────────┼──────────────┼───────────┼─────────────────┤
│ 2023-07-01 00:00:00 │ 0.12.0       │        40 │            7743 │
│ 2023-07-01 00:00:00 │ 5.0.0        │      5062 │           45781 │
│ 2023-07-01 00:00:00 │ 6.0.1.dev58  │       134 │             134 │
│ 2023-07-01 00:00:00 │ 5.1.0        │      7904 │           34213 │
│ 2023-07-01 00:00:00 │ 5.1.0.dev465 │         2 │             118 │
│ 2023-07-01 00:00:00 │ 0.4.1        │        17 │            4242 │
│ 2023-07-01 00:00:00 │ 1.0.0        │        32 │           36975 │
│ 2023-07-01 00:00:00 │ 3.0.1        │        33 │            1441 │
│ 2023-07-01 00:00:00 │ 6.0.1.dev33  │       172 │             172 │
│ 2023-07-01 00:00:00 │ v0.6.0       │        21 │            4563 │
│ …                   │ …            │         … │               … │
└─────────────────────┴──────────────┴───────────┴─────────────────┘

In [7]:
fig = px.line(
    t.to_pandas(),
    x="timestamp",
    y="total_downloads",
    color="version",
    title="Downloads",
)

In [8]:
fig

In [9]:
t_mod = clean_data(ibis.read_parquet("data/pypi/modin/*.parquet"))
t_mod = t_mod.drop("project").unpack("file").unpack("details")
t_mod = (
    t_mod.group_by(
        [
            ibis._.timestamp.truncate("D").name("timestamp"),
            ibis._.country_code,
            ibis._.version,
            ibis._.python,
            ibis._.system["name"].name("system"),
        ]
    )
    .agg(
        ibis._.count().name("downloads"),
    )
    .order_by(ibis._.timestamp.desc())
    .mutate(
        total_downloads=ibis._.downloads.sum().over(
            rows=(0, None),
            group_by=["country_code", "version", "python", "system"],
            order_by=ibis._.timestamp.desc(),
        )
    )
    .order_by(ibis._.timestamp.desc())
)

In [10]:
t_pola = clean_data(ibis.read_parquet("data/pypi/polars/*.parquet"))
t_pola = t_pola.drop("project").unpack("file").unpack("details")
t_pola = (
    t_pola.group_by(
        [
            ibis._.timestamp.truncate("D").name("timestamp"),
            ibis._.country_code,
            ibis._.version,
            ibis._.python,
            ibis._.system["name"].name("system"),
        ]
    )
    .agg(
        ibis._.count().name("downloads"),
    )
    .order_by(ibis._.timestamp.desc())
    .mutate(
        total_downloads=ibis._.downloads.sum().over(
            rows=(0, None),
            group_by=["country_code", "version", "python", "system"],
            order_by=ibis._.timestamp.desc(),
        )
    )
    .order_by(ibis._.timestamp.desc())
)

In [11]:
t_pola.schema()

ibis.Schema {
  timestamp        timestamp
  country_code     string
  version          string
  python           string
  system           string
  downloads        int64
  total_downloads  int64
}

In [12]:
t = (
    t_pola.group_by([ibis._.timestamp.truncate("M").name("timestamp"), ibis._.version])
    .agg(ibis._.downloads.sum().name("downloads"))
    .order_by(ibis._.timestamp.desc())
    .mutate(
        ibis._.downloads.sum()
        .over(rows=(0, None), group_by="version", order_by=ibis._.timestamp.desc())
        .name("total_downloads")
    )
    .order_by(ibis._.timestamp.desc())
)

In [13]:
fig = px.line(
    t.to_pandas(),
    x="timestamp",
    y="total_downloads",
    color="version",
    title="Downloads",
)

In [14]:
fig

In [15]:
t = (
    t_mod.group_by([ibis._.timestamp.truncate("M").name("timestamp"), ibis._.version])
    .agg(ibis._.downloads.sum().name("downloads"))
    .order_by(ibis._.timestamp.desc())
    .mutate(
        ibis._.downloads.sum()
        .over(rows=(0, None), group_by="version", order_by=ibis._.timestamp.desc())
        .name("total_downloads")
    )
    .order_by(ibis._.timestamp.desc())
)

In [16]:
fig = px.line(
    t.to_pandas(),
    x="timestamp",
    y="total_downloads",
    color="version",
    title="Downloads",
)

In [17]:
fig

In [18]:
t = (
    downloads.group_by(
        [ibis._.timestamp.truncate("W").name("timestamp"), ibis._.version]
    )
    .agg(ibis._.downloads.sum().name("downloads"))
    .order_by(ibis._.timestamp.desc())
    .mutate(
        ibis._.downloads.sum()
        .over(rows=(0, None), group_by="version", order_by=ibis._.timestamp.desc())
        .name("total_downloads")
    )
    .order_by(ibis._.timestamp.desc())
)

In [19]:
fig = px.line(
    t.to_pandas(), x="timestamp", y="downloads", color="version", title="Downloads"
)

In [20]:
fig

In [21]:
t = (
    downloads.group_by([ibis._.timestamp.truncate("M").name("timestamp")])
    .agg(ibis._.downloads.sum().name("downloads"))
    .order_by(ibis._.timestamp.desc())
    .mutate(
        ibis._.downloads.sum()
        .over(rows=(0, None), order_by=ibis._.timestamp.desc())
        .name("total_downloads")
    )
    .order_by(ibis._.timestamp.desc())
)

In [22]:
fig = px.line(t.to_pandas(), x="timestamp", y="downloads", title="Downloads")

In [23]:
fig

In [24]:
t = (
    t_mod.group_by([ibis._.timestamp.truncate("M").name("timestamp")])
    .agg(ibis._.downloads.sum().name("downloads"))
    .order_by(ibis._.timestamp.desc())
    .mutate(
        ibis._.downloads.sum()
        .over(rows=(0, None), order_by=ibis._.timestamp.desc())
        .name("total_downloads")
    )
    .order_by(ibis._.timestamp.desc())
)

In [25]:
fig = px.line(t.to_pandas(), x="timestamp", y="downloads", title="Downloads")

In [ ]:
fig

In [ ]:
t = (
    t_pola.group_by([ibis._.timestamp.truncate("M").name("timestamp")])
    .agg(ibis._.downloads.sum().name("downloads"))
    .order_by(ibis._.timestamp.desc())
    .mutate(
        ibis._.downloads.sum()
        .over(rows=(0, None), order_by=ibis._.timestamp.desc())
        .name("total_downloads")
    )
    .order_by(ibis._.timestamp.desc())
)

In [ ]:
fig = px.line(t.to_pandas(), x="timestamp", y="downloads", title="Downloads")

In [ ]:
fig

In [ ]:
fig = px.line(stars.to_pandas(), x="starred_at", y="total_stars", title="Stars")

In [ ]:
fig

In [ ]:
fig = px.line(pulls.to_pandas(), x="created_at", y="total_pulls", title="Pulls")

In [ ]:
fig

In [ ]:
downloads

In [ ]:
downloads.filter(ibis._.timestamp > datetime.now() - timedelta(days=7)).order_by(
    ibis._.downloads.desc()
)

In [ ]:
print(
    f"""
    Total downloads: {downloads.downloads.sum().to_pandas():,}
    Total stars: {stars.count().to_pandas():,}
    Total pulls: {pulls.count().to_pandas():,}
    """
)

In [ ]:
import pandas as pd

In [ ]:
downloads = pd.read_parquet("data/pypi/polars/file_downloads.parquet")

In [ ]:
downloads

In [ ]:
downloads.info()

In [ ]:
downloads.groupby("country_code").count().sort_values("file", ascending=False)

In [ ]:
forks